In [11]:
!pip install openml

In [12]:
import openml
import pandas as pd

# Loading data

In [13]:
breast_cancer_dataset = openml.datasets.get_dataset('breast-cancer')
X_breast_cancer, y_breast_cancer, categorical_indicator_breast_cancer, attribute_names_breast_cancer = (
    breast_cancer_dataset.get_data(
        dataset_format='dataframe',
        target=breast_cancer_dataset.default_target_attribute
    )
)

df_breast_cancer = pd.concat([X_breast_cancer, y_breast_cancer], axis=1)

adult_dataset = openml.datasets.get_dataset('adult')
X_adult, y_adult, categorical_indicator_adult, attribute_names_adult = (
    adult_dataset.get_data(
        dataset_format='dataframe',
        target=adult_dataset.default_target_attribute
    )
)

df_adult = pd.concat([X_adult, y_adult], axis=1)

madelon_dataset = openml.datasets.get_dataset('madelon')
X_madelon, y_madelon, categorical_indicator_madelon, attribute_names_madelon = (
    madelon_dataset.get_data(
        dataset_format='dataframe',
        target=madelon_dataset.default_target_attribute
    )
)

df_madelon = pd.concat([X_madelon, y_madelon], axis=1)


phoneme_dataset = openml.datasets.get_dataset('phoneme')
X_phoneme, y_phoneme, categorical_indicator_phoneme, attribute_names_phoneme = (
    phoneme_dataset.get_data(
        dataset_format='dataframe',
        target=phoneme_dataset.default_target_attribute
    )
)

df_phoneme = pd.concat([X_phoneme, y_phoneme], axis=1)

amazon_dataset = openml.datasets.get_dataset('amazon_employee_access')
X_amazon, y_amazon, categorical_indicator_amazon, attribute_names_amazon = (
    amazon_dataset.get_data(
        dataset_format='dataframe',
        target=amazon_dataset.default_target_attribute
    )
)

df_amazon = pd.concat([X_amazon, y_amazon], axis=1)

ozone_dataset = openml.datasets.get_dataset('ozone-level-8hr')
X_ozone, y_ozone, categorical_indicator_ozone, attribute_names_ozone = (
    ozone_dataset.get_data(
        dataset_format='dataframe',
        target=ozone_dataset.default_target_attribute
    )
)

df_ozone = pd.concat([X_ozone, y_ozone], axis=1)


# Finding hyperparameters


In [14]:
import numpy as np

import json
import ast

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
import lightgbm as lgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from scipy import stats


In [15]:
model_configs = {}

models = {
    'LogisticRegression': LogisticRegression,
    'RandomForestClassifier': RandomForestClassifier,
    'DecisionTreeClassifier': DecisionTreeClassifier,
    'XGBClassifier': xgb.XGBClassifier,
    'LGBMClassifier': lgb.LGBMClassifier
}

NUM_RANDOM_CONFIGS = 49 # plus default

# --- Logistic Regression ---
def lr_random_param_sampler(num_configs=NUM_RANDOM_CONFIGS, random_state=42):
    rng = np.random.default_rng(random_state)
    configs = []

    for _ in range(num_configs):


        solver = rng.choice(['liblinear', 'saga', 'lbfgs'])

        if solver == 'liblinear':
            penalty = rng.choice(['l1', 'l2'])
        elif solver == 'saga':
            penalty = rng.choice(['l1', 'l2', 'elasticnet', None])
        else:  # lbfgs
            penalty = rng.choice(['l2', None])

        config = {
            'solver': solver,
            'penalty': penalty,
            'class_weight': 'balanced',
            'random_state': 42
        }

        if penalty is not None:
            config['C'] = float(stats.loguniform(1e-4, 1e4).rvs(random_state=rng))

        if penalty == 'elasticnet':
            config['l1_ratio'] = float(rng.uniform(0.0, 1.0))

        configs.append(config)

    return configs

model_configs['LogisticRegression'] = {
    'default_params': {'solver': 'liblinear', 'random_state': 42},
    'random_param_sampler': lr_random_param_sampler
}

# --- RandomForestClassifier ---
def rfc_random_param_sampler():
    configs = []
    for i in range(NUM_RANDOM_CONFIGS):
        rng_np = np.random.RandomState(i)

        max_depth_val = stats.randint.rvs(5, 21, random_state=i)
        if rng_np.rand() < 0.2: # 20% chance of None
            max_depth_val = None

        config = {
            'n_estimators': stats.randint.rvs(50, 201, random_state=i),
            'max_depth': max_depth_val,
            'min_samples_split': stats.randint.rvs(2, 21, random_state=i),
            'min_samples_leaf': stats.randint.rvs(1, 11, random_state=i),
            'criterion': rng_np.choice(['gini', 'entropy']),
            'random_state': 42
        }
        configs.append(config)
    return configs

model_configs['RandomForestClassifier'] = {
    'default_params': {'random_state': 42},
    'random_param_sampler': rfc_random_param_sampler
}

# --- DecisionTreeClassifier ---
def dtc_random_param_sampler():
    configs = []
    for i in range(NUM_RANDOM_CONFIGS):
        rng_np = np.random.RandomState(i)

        max_depth_val = stats.randint.rvs(5, 21, random_state=i)
        if rng_np.rand() < 0.2:
            max_depth_val = None

        config = {
            'max_depth': max_depth_val,
            'min_samples_split': stats.randint.rvs(2, 21, random_state=i),
            'min_samples_leaf': stats.randint.rvs(1, 11, random_state=i),
            'criterion': rng_np.choice(['gini', 'entropy']),
            'random_state': 42
        }
        configs.append(config)
    return configs

model_configs['DecisionTreeClassifier'] = {
    'default_params': {'random_state': 42},
    'random_param_sampler': dtc_random_param_sampler
}

# --- XGBClassifier ---
def xgb_random_param_sampler():
    configs = []
    for i in range(NUM_RANDOM_CONFIGS):
        rng_np = np.random.RandomState(i)

        config = {
            'n_estimators': stats.randint.rvs(50, 201, random_state=i),
            'learning_rate': float(stats.loguniform.rvs(0.01, 0.3, random_state=i)),
            'max_depth': stats.randint.rvs(3, 11, random_state=i),
            'subsample': float(stats.uniform.rvs(0.6, 0.4, random_state=i)), # 0.6 to 1.0 (0.4 is range)
            'colsample_bytree': float(stats.uniform.rvs(0.6, 0.4, random_state=i)),
            'gamma': float(stats.uniform.rvs(0, 0.5, random_state=i)),
            'random_state': 42,
            'use_label_encoder': False,
            'eval_metric': 'logloss'
        }
        configs.append(config)
    return configs

model_configs['XGBClassifier'] = {
    'default_params': {'random_state': 42, 'use_label_encoder': False, 'eval_metric': 'logloss'},
    'random_param_sampler': xgb_random_param_sampler
}

# --- LGBMClassifier ---
def lgbm_random_param_sampler():
    configs = []
    for i in range(NUM_RANDOM_CONFIGS):
        rng_np = np.random.RandomState(i)

        config = {
            'n_estimators': stats.randint.rvs(50, 201, random_state=i),
            'learning_rate': float(stats.loguniform.rvs(0.01, 0.3, random_state=i)),
            'num_leaves': stats.randint.rvs(20, 101, random_state=i),
            'max_depth': stats.randint.rvs(5, 21, random_state=i),
            'min_child_samples': stats.randint.rvs(20, 101, random_state=i),
            'subsample': float(stats.uniform.rvs(0.6, 0.4, random_state=i)),
            'colsample_bytree': float(stats.uniform.rvs(0.6, 0.4, random_state=i)),
            'random_state': 42,
            # 'use_label_encoder': False,
            'eval_metric': 'binary_logloss'
        }
        configs.append(config)
    return configs

model_configs['LGBMClassifier'] = {
    'default_params': {'random_state': 42,  'eval_metric': 'binary_logloss'},
    'random_param_sampler': lgbm_random_param_sampler
}

print("Model configurations defined successfully.")


all_experiment_configs = []

# Iterate through each model and its configurations
for model_name, config_info in model_configs.items():
    i=1
    default_params = config_info['default_params'].copy()
    all_experiment_configs.append({
        'name': f"{model_name}_{i}",
        'class': models[model_name],
        'params': default_params
    })

    if 'random_param_sampler' in config_info:
        random_params_list = config_info['random_param_sampler']()
        for random_params in random_params_list:
            i+=1
            all_experiment_configs.append({
                'name': f"{model_name}_{i}",
                'class': models[model_name],
                'params': random_params
            })

print(f"Created all_experiment_configs with {len(all_experiment_configs)} total configurations.")




Model configurations defined successfully.
Created all_experiment_configs with 250 total configurations.


# Training

In [16]:
processed_datasets = {
    'breast_cancer': {
        'X': X_breast_cancer,
        'y': y_breast_cancer,
        'categorical_indicator': categorical_indicator_breast_cancer,
        'attribute_names': attribute_names_breast_cancer
    },
    'adult': {
        'X': X_adult,
        'y': y_adult,
        'categorical_indicator': categorical_indicator_adult,
        'attribute_names': attribute_names_adult
    },
    'madelon': {
        'X': X_madelon,
        'y': y_madelon,
        'categorical_indicator': categorical_indicator_madelon,
        'attribute_names': attribute_names_madelon
    },
    'phoneme': {
        'X': X_phoneme,
        'y': y_phoneme,
        'categorical_indicator': categorical_indicator_phoneme,
        'attribute_names': attribute_names_phoneme
    },
    'amazon_employee_access': {
        'X': X_amazon,
        'y': y_amazon,
        'categorical_indicator': categorical_indicator_amazon,
        'attribute_names': attribute_names_amazon
    },
    'ozone_level_8hr': {
        'X': X_ozone,
        'y': y_ozone,
        'categorical_indicator': categorical_indicator_ozone,
        'attribute_names': attribute_names_ozone
    }
}


In [17]:
def encode_binary_target(y):
    """
    Ensures binary target is encoded as {0, 1}.
    Returns encoded y and the mapping.
    """
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)

    if len(le.classes_) != 2:
        raise ValueError(f"Target is not binary: {le.classes_}")

    mapping = dict(zip(le.classes_, [0, 1]))
    return y_encoded, mapping


In [18]:
target_mappings = {}

for name, data in processed_datasets.items():
    y_encoded, mapping = encode_binary_target(data['y'])

    data['y'] = y_encoded
    target_mappings[name] = mapping


In [19]:
def evaluate_model_on_datasets(model_class, params, datasets):
    total_balanced_accuracy = 0
    evaluation_results = {}

    for dataset_name, data in datasets.items():
        X = data['X'].copy()
        y = data['y'].copy()
        categorical_indicator = data['categorical_indicator']
        attribute_names = data['attribute_names']

        # Categorical and numerical features
        categorical_features = [attr_name for i, attr_name in enumerate(attribute_names) if categorical_indicator[i]]
        numerical_features = [attr_name for i, attr_name in enumerate(attribute_names) if not categorical_indicator[i]]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

        # Preprocessing
        numerical_transformer = StandardScaler()
        categorical_transformer = OneHotEncoder(handle_unknown='ignore')

        preprocessor = ColumnTransformer(
            transformers=[
                ('num', numerical_transformer, numerical_features),
                ('cat', categorical_transformer, categorical_features)
            ])

        model = model_class(**params)
        pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('classifier', model)])

        # Train and test
        pipeline.fit(X_train, y_train)

        y_pred = pipeline.predict(X_test)

        balanced_acc = balanced_accuracy_score(y_test, y_pred)
        total_balanced_accuracy += balanced_acc
        evaluation_results[dataset_name] = balanced_acc

    return total_balanced_accuracy, evaluation_results

In [20]:
# Creating dataframe with results
results_list = []

for config in all_experiment_configs:
    model_name = config['name']
    model_class = config['class']
    params = config['params']

    print(f"Evaluating {model_name} with parameters: {params}")

    serializable_params = {}
    for k, v in params.items():
        if isinstance(v, (np.integer, np.floating, np.bool_)):
            serializable_params[k] = v.item()
        elif isinstance(v, np.ndarray):
            serializable_params[k] = v.tolist()
        elif isinstance(v, (str, int, float, bool, type(None))):
            serializable_params[k] = v
        else:
            serializable_params[k] = str(v) # Fallback for other types

    total_bal_acc, dataset_bal_accs = evaluate_model_on_datasets(model_class, serializable_params, processed_datasets)

    results_list.append({
        'Model Name': model_name,
        'Hyperparameters': json.dumps(serializable_params),
        'Total Balanced Accuracy': total_bal_acc,
        **{f'Balanced Accuracy ({ds})': acc for ds, acc in dataset_bal_accs.items()}
    })

results_df = pd.DataFrame(results_list)

results_df.head()

Evaluating LogisticRegression_1 with parameters: {'solver': 'liblinear', 'random_state': 42}
Evaluating LogisticRegression_2 with parameters: {'solver': np.str_('liblinear'), 'penalty': np.str_('l2'), 'class_weight': 'balanced', 'random_state': 42, 'C': 0.3243601689935789}
Evaluating LogisticRegression_3 with parameters: {'solver': np.str_('saga'), 'penalty': None, 'class_weight': 'balanced', 'random_state': 42}


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Evaluating LogisticRegression_4 with parameters: {'solver': np.str_('liblinear'), 'penalty': np.str_('l2'), 'class_weight': 'balanced', 'random_state': 42, 'C': 0.0005667855677111899}
Evaluating LogisticRegression_5 with parameters: {'solver': np.str_('saga'), 'penalty': None, 'class_weight': 'balanced', 'random_state': 42}


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Evaluating LogisticRegression_6 with parameters: {'solver': np.str_('lbfgs'), 'penalty': None, 'class_weight': 'balanced', 'random_state': 42}


...

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Info] Number of positive: 1110, number of negative: 2672
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000215 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 3782, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.293496 -> initscore=-0.878467
[LightGBM] [Info] Start training from score -0.878467
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Info] Number of positive: 21610, number of negative: 1328
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.064106 seconds.
You can set `force_row_w

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Info] Number of positive: 112, number of negative: 1661
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002459 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10608
[LightGBM] [Info] Number of data points in the train set: 1773, number of used features: 72
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.063170 -> initscore=-2.696676
[LightGBM] [Info] Start training from score -2.696676
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightG

,Model Name,Hyperparameters,Total Balanced Accuracy,Balanced Accuracy (breast_cancer),Balanced Accuracy (adult),Balanced Accuracy (madelon),Balanced Accuracy (phoneme),Balanced Accuracy (amazon_employee_access),Balanced Accuracy (ozone_level_8hr)
0,LogisticRegression_1,"{""solver"": ""liblinear"", ""random_state"": 42}",3.807844,0.595513,0.774376,0.557692,0.642635,0.575169,0.662459
1,LogisticRegression_2,"{""solver"": ""liblinear"", ""penalty"": ""l2"", ""clas...",4.391050,0.663462,0.824855,0.557692,0.732977,0.792069,0.819995
2,LogisticRegression_3,"{""solver"": ""saga"", ""penalty"": null, ""class_wei...",4.322948,0.660897,0.825177,0.557692,0.732977,0.733821,0.812383
3,LogisticRegression_4,"{""solver"": ""liblinear"", ""penalty"": ""l2"", ""clas...",4.183298,0.664103,0.804975,0.594872,0.737681,0.653553,0.728115
4,LogisticRegression_5,"{""solver"": ""saga"", ""penalty"": null, ""class_wei...",4.322948,0.660897,0.825177,0.557692,0.732977,0.733821,0.812383


In [21]:
results_df

,Model Name,Hyperparameters,Total Balanced Accuracy,Balanced Accuracy (breast_cancer),Balanced Accuracy (adult),Balanced Accuracy (madelon),Balanced Accuracy (phoneme),Balanced Accuracy (amazon_employee_access),Balanced Accuracy (ozone_level_8hr)
0,LogisticRegression_1,"{""solver"": ""liblinear"", ""random_state"": 42}",3.807844,0.595513,0.774376,0.557692,0.642635,0.575169,0.662459
1,LogisticRegression_2,"{""solver"": ""liblinear"", ""penalty"": ""l2"", ""clas...",4.391050,0.663462,0.824855,0.557692,0.732977,0.792069,0.819995
2,LogisticRegression_3,"{""solver"": ""saga"", ""penalty"": null, ""class_wei...",4.322948,0.660897,0.825177,0.557692,0.732977,0.733821,0.812383
3,LogisticRegression_4,"{""solver"": ""liblinear"", ""penalty"": ""l2"", ""clas...",4.183298,0.664103,0.804975,0.594872,0.737681,0.653553,0.728115
4,LogisticRegression_5,"{""solver"": ""saga"", ""penalty"": null, ""class_wei...",4.322948,0.660897,0.825177,0.557692,0.732977,0.733821,0.812383
...,...,...,...,...,...,...,...,...,...
245,LGBMClassifier_46,"{""n_estimators"": 70, ""learning_rate"": 0.171065...",4.198293,0.625641,0.781369,0.762821,0.854382,0.522037,0.652042
246,LGBMClassifier_47,"{""n_estimators"": 181, ""learning_rate"": 0.28899...",4.006975,0.500000,0.763638,0.696154,0.855594,0.552769,0.638821
247,LGBMClassifier_48,"{""n_estimators"": 119, ""learning_rate"": 0.14381...",4.067785,0.500000,0.775993,0.742308,0.858228,0.528797,0.662459
248,LGBMClassifier_49,"{""n_estimators"": 185, ""learning_rate"": 0.01471...",4.103028,0.592949,0.761189,0.808974,0.826716,0.510437,0.602764


In [22]:
#Sorting by 'total Balanced Accuracy', we could use mean from other scores, but it doesn't matter
results_df_sorted = results_df.sort_values(
    by='Total Balanced Accuracy',
    ascending=False
).reset_index(drop=True)

results_df_sorted.to_csv('results_df_sorted.csv', index=False)

In [24]:
# Finding top 50 sets, but at least 3 for each type of models
# Also we don't want sets that had the same reults
results_df_sorted['Model Type'] = results_df_sorted['Model Name'].apply(lambda x: '_'.join(x.split('_')[:-1]))

score_cols = [col for col in results_df_sorted.columns if 'Balanced Accuracy' in col]


df_unique = results_df_sorted.drop_duplicates(subset=['Model Type'] + score_cols, keep='first')

selected_indices = set()
min_per_type = 3
target_count = 50

# Top 3 for each type
for m_type in df_unique['Model Type'].unique():
    type_subset = df_unique[df_unique['Model Type'] == m_type]
    top_3 = type_subset.head(min_per_type)
    selected_indices.update(top_3.index.tolist())

# Complement to 50
current_count = len(selected_indices)
needed = target_count - current_count

if needed > 0:
    remaining = df_unique[~df_unique.index.isin(selected_indices)]
    best_rest = remaining.head(needed)
    selected_indices.update(best_rest.index.tolist())

final_df = df_unique.loc[list(selected_indices)]
final_df = final_df.sort_values(by='Total Balanced Accuracy', ascending=False)


In [27]:
def get_full_class_path(model_name):
        if "LogisticRegression" in model_name:
            return "sklearn.linear_model.LogisticRegression"
        elif "LGBMClassifier" in model_name:
            return "lightgbm.LGBMClassifier"
        elif "RandomForest" in model_name:
            return "sklearn.ensemble.RandomForestClassifier"
        elif "DecisionTree" in model_name:
            return "sklearn.tree.DecisionTreeClassifier"
        elif "XGBClassifier" in model_name:
            return "xgboost.XGBClassifier"

output_list = []

for index, row in final_df.iterrows():

    raw_params = row['Hyperparameters']
    try:

        params_dict = json.loads(raw_params)
    except:
        try:
            params_dict = ast.literal_eval(raw_params)
        except:
            params_dict = {}

    entry = {
        "name": row['Model Name'],
        "class": get_full_class_path(row['Model Name']),
        "params": params_dict
    }
    output_list.append(entry)

with open('model_portfolio.json', 'w') as f:
    json.dump(output_list, f, indent=2)

